In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
transaction = pd.read_csv('./data/transaction_new.csv')
customer = pd.read_csv('./data/customer.csv')

In [22]:
transaction

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price,total_price
0,2018-07-29,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832,54728,1,191247,189832
1,2018-07-30,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526,16193,1,145526,145526
2,2018-09-15,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696,53686,4,135174,540696
3,2018-11-01,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012,20228,1,271012,271012
4,2018-12-18,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753,55220,1,198753,198753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254580,2021-09-22,25121,66f02fa4-c62e-497a-8b6e-be239009b50a,484eecde-9f4a-42de-a8df-7ce6668101f4,Debit Card,Failed,0,NaN,0,2021-09-27T00:43:40.992167Z,-6.869535,111.077145,226820,30642,2,113410,226820
1254581,2021-12-09,34829,ccde6b83-ee42-4cce-b2cd-976574886eac,954e1211-4d45-4d0b-ba84-53e80119592c,OVO,Success,5010,AZ2022,10000,2021-12-12T07:52:44.803822Z,-8.677264,115.176947,359203,19394,1,354213,349203
1254582,2021-12-11,99645,5bcd3434-1dc6-44bb-ae63-fd19714cbd6c,e6e6c108-abd0-49a2-b0a5-691117f9cfe1,Credit Card,Success,2181,WEEKENDSERU,5000,2021-12-12T13:28:22.848703Z,-6.185498,106.831093,1158159,13079,4,288835,1153159
1254583,2019-12-18,87319,af7f291a-b01b-4bfd-89a9-237b16368b15,bb6e5426-a3f3-426a-b99d-79173eaa234b,Credit Card,Success,4503,AZ2022,0,2019-12-23T22:56:46.831720Z,-7.155977,110.891858,184143,11312,1,188646,184143


In [3]:
customer = customer.assign(first_join_date=pd.to_datetime(customer['first_join_date'])+timedelta(hours=7))
transaction = transaction.assign(created_at=pd.to_datetime(transaction['created_at'])+timedelta(hours=7))
transaction = transaction.assign(created_at = transaction['created_at'].dt.strftime('%Y-%m-%d'))
transaction = transaction.assign(created_at = pd.to_datetime(transaction['created_at']))
transaction = transaction.assign(total_price=transaction['total_amount'] - transaction['shipment_fee'])

In [23]:
paid_transaction = transaction[transaction.payment_status=='Success']

In [4]:
customer_date = customer[['customer_id', 'first_join_date']].sort_values('first_join_date').reset_index(drop=True)
customer_date

,customer_id,first_join_date
0,74089,2016-06-30 07:00:00
1,21734,2016-06-30 07:00:00
2,38119,2016-06-30 07:00:00
3,73773,2016-06-30 07:00:00
4,60006,2016-06-30 07:00:00
...,...,...
99995,73572,2022-07-31 07:00:00
99996,66776,2022-07-31 07:00:00
99997,96644,2022-07-31 07:00:00
99998,27995,2022-07-31 07:00:00


* ARPU
    * 매출(주 단위) / 주 단위로 누적(유입) 유저
* ARPPU
    * 매출(주 단위) / 주 단위로 누적 구매 유저

In [24]:
transaction_by_session = paid_transaction.groupby('session_id')[['created_at', 'customer_id', 'total_price']].max().sort_values('created_at').reset_index(drop=True)
transaction_by_session

,created_at,customer_id,total_price
0,2016-07-01,16228,318792
1,2016-07-01,74089,630795
2,2016-07-01,6183,568826
3,2016-07-02,933,237978
4,2016-07-02,73773,246670
...,...,...,...
815959,2022-08-01,81975,998856
815960,2022-08-01,19551,417340
815961,2022-08-01,83018,423480
815962,2022-08-01,89791,350247


In [25]:
data_start_date = pd.to_datetime('2016-06-30')
previous_start_date = pd.to_datetime('2016-07-11')
previous_end_date = previous_start_date + timedelta(days=6)

current_start_date = previous_start_date + timedelta(days=7)
current_end_date = previous_start_date + timedelta(days=13)

print(previous_start_date, previous_end_date)
print(current_start_date, current_end_date)

2016-07-11 00:00:00 2016-07-17 00:00:00
2016-07-18 00:00:00 2016-07-24 00:00:00


In [26]:
prev_customer = customer_date[(customer_date.first_join_date>=data_start_date)&
                              (customer_date.first_join_date<=previous_end_date)]

curr_customer = customer_date[(customer_date.first_join_date>=data_start_date)&
                              (customer_date.first_join_date<=current_end_date)]

In [27]:
prev_transaction = transaction_by_session[(transaction_by_session.created_at>=previous_start_date)&
                                          (transaction_by_session.created_at<=previous_end_date)]

curr_transaction = transaction_by_session[(transaction_by_session.created_at>=current_start_date)&
                                          (transaction_by_session.created_at<=current_end_date)]

prev_transaction_cum = transaction_by_session[(transaction_by_session.created_at>=data_start_date)&
                                              (transaction_by_session.created_at<=previous_end_date)]

curr_transaction_cum = transaction_by_session[(transaction_by_session.created_at>=data_start_date)&
                                              (transaction_by_session.created_at<=current_end_date)]

In [28]:
# ARPU

print('prev', prev_transaction.total_price.sum() / prev_customer.customer_id.nunique())
print('curr', curr_transaction.total_price.sum() / curr_customer.customer_id.nunique())

prev 63851.522077922076
curr 62858.37608318891


In [29]:
# ARPPU
print('prev', prev_transaction.total_price.sum() / prev_transaction_cum.customer_id.nunique())
print('curr', curr_transaction.total_price.sum() / curr_transaction_cum.customer_id.nunique())

prev 175591.6857142857
curr 184108.03553299492


* ARPU
    * 매출(월 단위) / 월 단위로 누적(유입) 유저
* ARPPU
    * 매출(주 단위) / 월 단위로 누적 구매 유저

In [30]:
customer_date = customer_date.assign(first_join_month = customer_date['first_join_date'].dt.strftime('%Y-%m'))
transaction_by_session = transaction_by_session.assign(purchase_month = transaction_by_session['created_at'].dt.strftime('%Y-%m'))

display(customer_date)
display(transaction_by_session)

,customer_id,first_join_date,first_join_month
0,74089,2016-06-30 07:00:00,2016-06
1,21734,2016-06-30 07:00:00,2016-06
2,38119,2016-06-30 07:00:00,2016-06
3,73773,2016-06-30 07:00:00,2016-06
4,60006,2016-06-30 07:00:00,2016-06
...,...,...,...
99995,73572,2022-07-31 07:00:00,2022-07
99996,66776,2022-07-31 07:00:00,2022-07
99997,96644,2022-07-31 07:00:00,2022-07
99998,27995,2022-07-31 07:00:00,2022-07


,created_at,customer_id,total_price,purchase_month
0,2016-07-01,16228,318792,2016-07
1,2016-07-01,74089,630795,2016-07
2,2016-07-01,6183,568826,2016-07
3,2016-07-02,933,237978,2016-07
4,2016-07-02,73773,246670,2016-07
...,...,...,...,...
815959,2022-08-01,81975,998856,2022-08
815960,2022-08-01,19551,417340,2022-08
815961,2022-08-01,83018,423480,2022-08
815962,2022-08-01,89791,350247,2022-08


In [31]:
FOR_USER = customer_date[customer_date['first_join_month']<='2017-01']
FOR_PU = transaction_by_session[transaction_by_session['purchase_month']<='2017-01']
FOR_REVENUE = transaction_by_session[transaction_by_session['purchase_month'].str.contains('2017')]

In [32]:
FOR_USER

,customer_id,first_join_date,first_join_month
0,74089,2016-06-30 07:00:00,2016-06
1,21734,2016-06-30 07:00:00,2016-06
2,38119,2016-06-30 07:00:00,2016-06
3,73773,2016-06-30 07:00:00,2016-06
4,60006,2016-06-30 07:00:00,2016-06
...,...,...,...
5540,87033,2017-01-31 07:00:00,2017-01
5541,37491,2017-01-31 07:00:00,2017-01
5542,3127,2017-01-31 07:00:00,2017-01
5543,87539,2017-01-31 07:00:00,2017-01


In [33]:
USER = FOR_USER.customer_id.nunique()
USER

5545

In [34]:
FOR_PU

,created_at,customer_id,total_price,purchase_month
0,2016-07-01,16228,318792,2016-07
1,2016-07-01,74089,630795,2016-07
2,2016-07-01,6183,568826,2016-07
3,2016-07-02,933,237978,2016-07
4,2016-07-02,73773,246670,2016-07
...,...,...,...,...
6071,2017-01-31,95749,379451,2017-01
6072,2017-01-31,68004,145704,2017-01
6073,2017-01-31,46070,123472,2017-01
6074,2017-01-31,86059,301780,2017-01


In [35]:
PU = FOR_PU.customer_id.nunique()
PU

2899

In [36]:
REVENUE = FOR_REVENUE.groupby('purchase_month')['total_price'].sum().to_frame().reset_index()
REVENUE

,purchase_month,total_price
0,2017-01,751383393
1,2017-02,790937818
2,2017-03,1032915810
3,2017-04,1059938716
4,2017-05,1267289808
5,2017-06,1195632027
6,2017-07,1727201517
7,2017-08,1681496555
8,2017-09,1825073155
9,2017-10,2051789976


In [37]:
# ARPPU

REVENUE[REVENUE['purchase_month']=='2017-01']['total_price'] / PU

0    259187.096585
Name: total_price, dtype: float64

In [41]:
784823105 / 2899

270722.0093135564

In [39]:
# ARPU

REVENUE[REVENUE['purchase_month']=='2017-01']['total_price'] / USER

0    135506.473039
Name: total_price, dtype: float64

In [40]:
784823105 / 5545

141537.07935076646